# Demo 1: Fallzahlen, Genesungen, und Todesfällen Vergleichen

## Importing Dependencies

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as wgts
from IPython.display import display
import numpy as np

## Loading Datasets

In [2]:
# test data read
covid_19_dataset = pd.read_csv('../dataset/cleaned_dataset.csv')
regions_dataset = pd.read_csv('../dataset/Countries by continents.csv')

## Creating Widgets

### Creating Countries & Continents Dropdown Menus

In [3]:




continent_dropdown_menu = wgts.Dropdown(
    options= regions_dataset['Continent'].unique(),
    description='Continent:',
    disabled=False,
)

continent_filter = regions_dataset['Continent'] == continent_dropdown_menu.value
countries = regions_dataset[continent_filter]
countries = countries['Country']

countries_dropdown_menu = wgts.Dropdown(
    options= countries,
    description='Country:',
    disabled=False,
)

### Creating Regional Category Radio Button

In [4]:
regional_category_radio = wgts.RadioButtons(
    options=['Country', 'Continent', 'World'],
    description='Regional Category:',
    disabled=False
)

### Creating Time Category Radio Button

In [5]:
time_category_radio = wgts.RadioButtons(
    options=['Daily', 'Monthly', 'Yearly', 'Complete'],
    description='Time Category:',
    disabled=False
)

### Creating Months, Years Dropdown Menus & Day DatePicker

In [6]:
import datetime
# minimum date value in dataset
dateMin = datetime.datetime.strptime(covid_19_dataset['Date'].min(), '%Y-%m-%d').date()
# maximum date value in dataset
dateMax = datetime.datetime.strptime(covid_19_dataset['Date'].max(), '%Y-%m-%d').date()

day_datepicker = wgts.DatePicker(
    description='Day:',
    disabled=False,
    value=dateMin,
    max=dateMax,
    min=dateMin
)

year_dropdown_menu = wgts.Dropdown(
    options= range(dateMin.year, dateMax.year + 1),
    description='Year:',
    disabled=True,
)

month_dropdown_menu = wgts.Dropdown(
    options= ["January", "February", "March", "April", "May", "June",
          "July", "August", "September", "October", "November", "December"],
    description='Month:',
    disabled=True,
)



### Creating Reload Button

In [7]:
reload_button = wgts.Button(
    description='Reload',
    disabled=False,
    button_style='info',
    tooltip='Reload',
    icon='check'
)

## Plotting Functions

### Country By Day

In [8]:
def plot_country_by_day(country:str, day:str):
    df = pd.DataFrame(covid_19_dataset)

    # Convert Date column to datetime
    df['Date'] = pd.to_datetime(df['Date'])

    # Target date
    target_date = day

    # 1. Get Country's data for the target date
    date_filter = df['Date'] == pd.to_datetime(target_date)
    target_df = df[date_filter]
    country_on_date = target_df[target_df['Country'] == country]
    country_date_values = country_on_date[['Confirmed', 'Recovered', 'Deaths']].values[0]

    # 2. Calculate average for Country across all days
    country_all_days = df[df['Country'] == country]
    country_avg_values = country_all_days[['Confirmed', 'Recovered', 'Deaths']].mean().values

    # 3. Calculate averages for all countries on the target date
    all_countries_date_avg = target_df[['Confirmed', 'Recovered', 'Deaths']].mean().values

    # Set up the plot
    categories = ['Confirmed', 'Recovered', 'Deaths']
    x = np.arange(len(categories))
    width = 0.25  # Narrower bars to fit three side by side

    fig, ax = plt.subplots(figsize=(8, 5))

    # Create the three sets of bars
    bar1 = ax.bar(x - width, country_date_values, width, label=f'{country} on {target_date}', color='#1f77b4')
    bar2 = ax.bar(x, country_avg_values, width, label=f'{country} Average (All Days)', color='#ff7f0e')
    bar3 = ax.bar(x + width, all_countries_date_avg, width, label=f'All Countries Average on {target_date}', color='#2ca02c')

    # Add labels, title and custom x-axis tick labels
    ax.set_xlabel('COVID-19 Metrics', fontsize=9)
    ax.set_ylabel('Number of Cases', fontsize=9)
    ax.set_title(f'COVID-19 Statistics Comparison for {country}\n{target_date}', fontsize=12)
    ax.set_xticks(x)
    ax.set_xticklabels(categories, fontsize=9)
    ax.legend(fontsize=9)

    # Add counts above bars
    def add_labels(bars):
        for bar in bars:
            height = bar.get_height()
            ax.annotate(f'{int(height):,}',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=10)

    add_labels(bar1)
    add_labels(bar2)
    add_labels(bar3)

    # Add grid lines for better readability
    ax.grid(axis='y', linestyle='--', alpha=0.7)

    # Print the numerical results for reference
    print(f"{country} values on {target_date}:")
    for category, value in zip(categories, country_date_values):
        print(f"  {category}: {value:.0f}")

    print(f"\n{country} average across all days:")
    for category, value in zip(categories, country_avg_values):
        print(f"  {category}: {value:.2f}")

    print(f"\nAll countries average on {target_date}:")
    for category, value in zip(categories, all_countries_date_avg):
        print(f"  {category}: {value:.2f}")

    plt.tight_layout()
    plt.show()

### Country By Month

In [9]:
def plot_country_by_month(country: str, month: str, year: str):
    df = pd.DataFrame(covid_19_dataset)
    
    # Convert Date column to datetime
    df['Date'] = pd.to_datetime(df['Date'])
    
    # Convert month and year to integer
    year_num = int(year)
    month_num = int(month)
    
    # Filter the dataset for the specified month and year
    month_filter = (df['Date'].dt.month == month_num) & (df['Date'].dt.year == year_num)
    month_df = df[month_filter]
    
    # 1. Get Country's data for the target month (averages)
    country_month_df = month_df[month_df['Country'] == country]
    country_month_avg = country_month_df[['Confirmed', 'Recovered', 'Deaths']].mean().values
    
    # 2. Calculate the country's average across all recorded days
    country_all_days = df[df['Country'] == country]
    country_avg_values = country_all_days[['Confirmed', 'Recovered', 'Deaths']].mean().values
    
    # 3. Calculate averages for all countries on the target month
    all_countries_month_avg = month_df[['Confirmed', 'Recovered', 'Deaths']].mean().values
    
    # Set up the plot
    categories = ['Confirmed', 'Recovered', 'Deaths']
    x = np.arange(len(categories))
    width = 0.25  # Narrower bars to fit three side by side
    
    fig, ax = plt.subplots(figsize=(8, 5))

    month_name = pd.to_datetime(f"{year}-{month}-01").strftime('%B')
    
    # Create the three sets of bars
    bar1 = ax.bar(x - width, country_month_avg, width, label=f'{country} in {month}/{year}', color='#1f77b4')
    bar2 = ax.bar(x, country_avg_values, width, label=f'{country} Average (All Time)', color='#ff7f0e')
    bar3 = ax.bar(x + width, all_countries_month_avg, width, label=f'All Countries Average in {month}/{year}', color='#2ca02c')
    
    # Add labels, title, and custom x-axis tick labels
    ax.set_xlabel('COVID-19 Metrics', fontsize=9)
    ax.set_ylabel('Number of Cases', fontsize=9)
    ax.set_title(f'COVID-19 Statistics Comparison for {country} ({month}/{year})', fontsize=12)
    ax.set_xticks(x)
    ax.set_xticklabels(categories, fontsize=9)
    ax.legend(fontsize=9)
    
    # Add counts above bars
    def add_labels(bars):
        for bar in bars:
            height = bar.get_height()
            ax.annotate(f'{int(height):,}',
                        xy=(bar.get_x() + bar.get_width() / 2, height),
                        xytext=(0, 3),  # 3 points vertical offset
                        textcoords="offset points",
                        ha='center', va='bottom', fontsize=10)
    
    add_labels(bar1)
    add_labels(bar2)
    add_labels(bar3)
    
    # Add grid lines for better readability
    ax.grid(axis='y', linestyle='--', alpha=0.7)
    
    # Print the numerical results for reference
    print(f"{country} values in {month}/{year}:")
    for category, value in zip(categories, country_month_avg):
        print(f"  {category}: {value:.0f}")
    
    print(f"\n{country} average across all recorded days:")
    for category, value in zip(categories, country_avg_values):
        print(f"  {category}: {value:.2f}")
    
    print(f"\nAll countries average in {month_name}/{year}:")
    for category, value in zip(categories, all_countries_month_avg):
        print(f"  {category}: {value:.2f}")
    
    plt.tight_layout()
    plt.show()

### Country By Year

In [10]:
def plot_country_by_year(country: str, year: str):
    df = pd.DataFrame(covid_19_dataset)
    
    # Convert Date column to datetime
    df['Date'] = pd.to_datetime(df['Date'])
    
    # Convert year to integer
    year = int(year)
    
    # Filter the dataset for the specified year
    year_filter = df['Date'].dt.year == year
    year_df = df[year_filter]
    
    # 1. Get Country's data for the target year (averages)
    country_year_df = year_df[year_df['Country'] == country]
    country_year_avg = country_year_df[['Confirmed', 'Recovered', 'Deaths']].mean().values
    
    # 2. Calculate the country's average across all recorded days
    country_all_days = df[df['Country'] == country]
    country_avg_values = country_all_days[['Confirmed', 'Recovered', 'Deaths']].mean().values
    
    # 3. Calculate averages for all countries in the target year
    all_countries_year_avg = year_df[['Confirmed', 'Recovered', 'Deaths']].mean().values
    
    # Set up the plot
    categories = ['Confirmed', 'Recovered', 'Deaths']
    x = np.arange(len(categories))
    width = 0.25  # Narrower bars to fit three side by side
    
    fig, ax = plt.subplots(figsize=(8, 5))
    
    # Create the three sets of bars
    bar1 = ax.bar(x - width, country_year_avg, width, label=f'{country} in {year}', color='#1f77b4')
    bar2 = ax.bar(x, country_avg_values, width, label=f'{country} Average (All Time)', color='#ff7f0e')
    bar3 = ax.bar(x + width, all_countries_year_avg, width, label=f'All Countries Average in {year}', color='#2ca02c')
    
    # Add labels, title, and custom x-axis tick labels
    ax.set_xlabel('COVID-19 Metrics', fontsize=9)
    ax.set_ylabel('Number of Cases', fontsize=9)
    ax.set_title(f'COVID-19 Statistics Comparison for {country} ({year})', fontsize=12)
    ax.set_xticks(x)
    ax.set_xticklabels(categories, fontsize=9)
    ax.legend(fontsize=9)
    
    # Add counts above bars
    def add_labels(bars):
        for bar in bars:
            height = bar.get_height()
            ax.annotate(f'{int(height):,}',
                        xy=(bar.get_x() + bar.get_width() / 2, height),
                        xytext=(0, 3),  # 3 points vertical offset
                        textcoords="offset points",
                        ha='center', va='bottom', fontsize=10)
    
    add_labels(bar1)
    add_labels(bar2)
    add_labels(bar3)
    
    # Add grid lines for better readability
    ax.grid(axis='y', linestyle='--', alpha=0.7)
    
    # Print the numerical results for reference
    print(f"{country} values in {year}:")
    for category, value in zip(categories, country_year_avg):
        print(f"  {category}: {value:.0f}")
    
    print(f"\n{country} average across all recorded days:")
    for category, value in zip(categories, country_avg_values):
        print(f"  {category}: {value:.2f}")
    
    print(f"\nAll countries average in {year}:")
    for category, value in zip(categories, all_countries_year_avg):
        print(f"  {category}: {value:.2f}")
    
    plt.tight_layout()
    plt.show()

### Country Total

In [11]:
def plot_country_by_total(country: str):
    df = pd.DataFrame(covid_19_dataset)
    
    # Convert Date column to datetime
    df['Date'] = pd.to_datetime(df['Date'])
    
    # Calculate the country's average across all recorded days
    country_all_days = df[df['Country'] == country]
    country_avg_values = country_all_days[['Confirmed', 'Recovered', 'Deaths']].mean().values
    
    # Calculate global average across all recorded days
    global_avg_values = df[['Confirmed', 'Recovered', 'Deaths']].mean().values
    
    # Set up the plot
    categories = ['Confirmed', 'Recovered', 'Deaths']
    x = np.arange(len(categories))
    width = 0.35  # Wider bars for clear comparison
    
    fig, ax = plt.subplots(figsize=(8, 5))
    
    # Create the two sets of bars
    bar1 = ax.bar(x - width/2, country_avg_values, width, label=f'{country} Average (All Time)', color='#1f77b4')
    bar2 = ax.bar(x + width/2, global_avg_values, width, label='Global Average (All Time)', color='#ff7f0e')
    
    # Add labels, title, and custom x-axis tick labels
    ax.set_xlabel('COVID-19 Metrics', fontsize=9)
    ax.set_ylabel('Number of Cases', fontsize=9)
    ax.set_title(f'COVID-19 Total Statistics Comparison for {country}', fontsize=12)
    ax.set_xticks(x)
    ax.set_xticklabels(categories, fontsize=9)
    ax.legend(fontsize=9)
    
    # Add counts above bars
    def add_labels(bars):
        for bar in bars:
            height = bar.get_height()
            ax.annotate(f'{int(height):,}',
                        xy=(bar.get_x() + bar.get_width() / 2, height),
                        xytext=(0, 3),  # 3 points vertical offset
                        textcoords="offset points",
                        ha='center', va='bottom', fontsize=10)
    
    add_labels(bar1)
    add_labels(bar2)
    
    # Add grid lines for better readability
    ax.grid(axis='y', linestyle='--', alpha=0.7)
    
    # Print the numerical results for reference
    print(f"{country} average across all recorded days:")
    for category, value in zip(categories, country_avg_values):
        print(f"  {category}: {value:.2f}")
    
    print(f"\nGlobal average across all recorded days:")
    for category, value in zip(categories, global_avg_values):
        print(f"  {category}: {value:.2f}")
    
    plt.tight_layout()
    plt.show()

### Continent By Day

In [12]:
def plot_continent_by_day(continent: str, day: str):
    df = pd.DataFrame(covid_19_dataset)

    countries = regions_dataset[regions_dataset['Continent'] == continent]
    countries = countries['Country']
    
    # Convert Date column to datetime
    df['Date'] = pd.to_datetime(df['Date'])
    
    # Target date
    target_date = pd.to_datetime(day)
    
    # 1. Get continent's data for the target date
    date_filter = df['Date'] == target_date
    target_df = df[date_filter]
    continent_on_date = target_df[target_df['Country'].isin(countries)]
    continent_date_values = continent_on_date[['Confirmed', 'Recovered', 'Deaths']].sum().values
    
    # 2. Calculate average for continent across all days
    continent_all_days = df[df['Country'].isin(countries)]
    continent_avg_values = continent_all_days[['Confirmed', 'Recovered', 'Deaths']].groupby(df['Date']).sum().mean().values
    
    # 3. Calculate averages for all countries on the target date
    all_countries_date_avg = target_df[['Confirmed', 'Recovered', 'Deaths']].mean().values
    
    # Set up the plot
    categories = ['Confirmed', 'Recovered', 'Deaths']
    x = np.arange(len(categories))
    width = 0.25  # Narrower bars to fit three side by side
    
    fig, ax = plt.subplots(figsize=(8, 5))
    
    # Create the three sets of bars
    bar1 = ax.bar(x - width, continent_date_values, width, label=f'{continent} on {target_date.date()}', color='#1f77b4')
    bar2 = ax.bar(x, continent_avg_values, width, label=f'{continent} Average (All Days)', color='#ff7f0e')
    bar3 = ax.bar(x + width, all_countries_date_avg, width, label=f'Global Average on {target_date.date()}', color='#2ca02c')
    
    # Add labels, title, and custom x-axis tick labels
    ax.set_xlabel('COVID-19 Metrics', fontsize=9)
    ax.set_ylabel('Number of Cases', fontsize=9)
    ax.set_title(f'COVID-19 Statistics Comparison for {continent} {target_date.date()}', fontsize=12)
    ax.set_xticks(x)
    ax.set_xticklabels(categories, fontsize=9)
    ax.legend(fontsize=9)
    
    # Add counts above bars
    def add_labels(bars):
        for bar in bars:
            height = bar.get_height()
            ax.annotate(f'{int(height):,}',
                        xy=(bar.get_x() + bar.get_width() / 2, height),
                        xytext=(0, 3),  # 3 points vertical offset
                        textcoords="offset points",
                        ha='center', va='bottom', fontsize=10)
    
    add_labels(bar1)
    add_labels(bar2)
    add_labels(bar3)
    
    # Add grid lines for better readability
    ax.grid(axis='y', linestyle='--', alpha=0.7)
    
    # Print the numerical results for reference
    print(f"{continent} values on {target_date.date()}:")
    for category, value in zip(categories, continent_date_values):
        print(f"  {category}: {value:.0f}")
    
    print(f"\n{continent} average across all days:")
    for category, value in zip(categories, continent_avg_values):
        print(f"  {category}: {value:.2f}")
    
    print(f"\nGlobal average on {target_date.date()}:")
    for category, value in zip(categories, all_countries_date_avg):
        print(f"  {category}: {value:.2f}")
    
    plt.tight_layout()
    plt.show()

### Continent By Month

In [13]:
def plot_continent_by_month(continent: str, month: str, year: str):
    df = pd.DataFrame(covid_19_dataset)

    # Get countries in the specified continent
    countries = regions_dataset[regions_dataset['Continent'] == continent]
    countries = countries['Country']
    
    # Convert Date column to datetime
    df['Date'] = pd.to_datetime(df['Date'])
    
    # Extract month and year for filtering
    month_num = int(month)
    year_num = int(year)
    
    # Add month and year columns for easy filtering
    df['Month'] = df['Date'].dt.month
    df['Year'] = df['Date'].dt.year
    
    # 1. Get continent's data for the target month
    month_filter = (df['Month'] == month_num) & (df['Year'] == year_num)
    target_df = df[month_filter]
    continent_in_month = target_df[target_df['Country'].isin(countries)]
    
    # Calculate the total for the continent for the month
    continent_month_values = continent_in_month[['Confirmed', 'Recovered', 'Deaths']].sum().values
    
    # 2. Calculate average for continent across all months
    continent_all_time = df[df['Country'].isin(countries)]
    # Group by month and year to get monthly averages
    continent_avg_values = (continent_all_time[['Confirmed', 'Recovered', 'Deaths']]
                           .groupby([df['Year'], df['Month']])
                           .sum()
                           .mean()
                           .values)
    
    # 3. Calculate global average for the target month
    all_countries_month_avg = target_df[['Confirmed', 'Recovered', 'Deaths']].mean().values
    
    # Set up the plot
    categories = ['Confirmed', 'Recovered', 'Deaths']
    x = np.arange(len(categories))
    width = 0.25  # Narrower bars to fit three side by side
    
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Month name for display
    month_name = pd.to_datetime(f"{year}-{month}-01").strftime('%B')
    
    # Create the three sets of bars
    bar1 = ax.bar(x - width, continent_month_values, width, 
                 label=f'{continent} in {month_name} {year}', color='#1f77b4')
    bar2 = ax.bar(x, continent_avg_values, width, 
                 label=f'{continent} Monthly Average (All Time)', color='#ff7f0e')
    bar3 = ax.bar(x + width, all_countries_month_avg, width, 
                 label=f'Global Average in {month_name} {year}', color='#2ca02c')
    
    # Add labels, title, and custom x-axis tick labels
    ax.set_xlabel('COVID-19 Metrics', fontsize=10)
    ax.set_ylabel('Number of Cases', fontsize=10)
    ax.set_title(f'COVID-19 Statistics for {continent} in {month_name} {year}', fontsize=14)
    ax.set_xticks(x)
    ax.set_xticklabels(categories, fontsize=10)
    ax.legend(fontsize=9)
    
    # Add counts above bars
    def add_labels(bars):
        for bar in bars:
            height = bar.get_height()
            ax.annotate(f'{int(height):,}',
                        xy=(bar.get_x() + bar.get_width() / 2, height),
                        xytext=(0, 3),  # 3 points vertical offset
                        textcoords="offset points",
                        ha='center', va='bottom', fontsize=10)
    
    add_labels(bar1)
    add_labels(bar2)
    add_labels(bar3)
    
    # Add grid lines for better readability
    ax.grid(axis='y', linestyle='--', alpha=0.7)
    
    # Print the numerical results for reference
    print(f"{continent} values in {month_name} {year}:")
    for category, value in zip(categories, continent_month_values):
        print(f"  {category}: {value:.0f}")
    
    print(f"\n{continent} monthly average across all time:")
    for category, value in zip(categories, continent_avg_values):
        print(f"  {category}: {value:.2f}")
    
    print(f"\nGlobal average in {month_name} {year}:")
    for category, value in zip(categories, all_countries_month_avg):
        print(f"  {category}: {value:.2f}")
    
    plt.tight_layout()
    plt.show()

### Continent By Year

In [14]:
def plot_continent_by_year(continent: str, year: str):
    df = pd.DataFrame(covid_19_dataset)

    # Get countries in the specified continent
    countries = regions_dataset[regions_dataset['Continent'] == continent]
    countries = countries['Country']
    
    # Convert Date column to datetime
    df['Date'] = pd.to_datetime(df['Date'])
    
    # Extract year for filtering
    year_num = int(year)
    
    # Add year column for easy filtering
    df['Year'] = df['Date'].dt.year
    
    # 1. Get continent's data for the target year
    year_filter = (df['Year'] == year_num)
    target_df = df[year_filter]
    continent_in_year = target_df[target_df['Country'].isin(countries)]
    
    # Calculate the total for the continent for the year
    continent_year_values = continent_in_year[['Confirmed', 'Recovered', 'Deaths']].sum().values
    
    # 2. Calculate average for continent across all years
    continent_all_time = df[df['Country'].isin(countries)]
    # Group by year to get yearly averages
    continent_avg_values = (continent_all_time[['Confirmed', 'Recovered', 'Deaths']]
                           .groupby([df['Year']])
                           .sum()
                           .mean()
                           .values)
    
    # 3. Calculate global average for the target year
    all_countries_year_avg = target_df[['Confirmed', 'Recovered', 'Deaths']].mean().values
    
    # Set up the plot
    categories = ['Confirmed', 'Recovered', 'Deaths']
    x = np.arange(len(categories))
    width = 0.25  # Narrower bars to fit three side by side
    
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Create the three sets of bars
    bar1 = ax.bar(x - width, continent_year_values, width, 
                 label=f'{continent} in {year}', color='#1f77b4')
    bar2 = ax.bar(x, continent_avg_values, width, 
                 label=f'{continent} Yearly Average (All Time)', color='#ff7f0e')
    bar3 = ax.bar(x + width, all_countries_year_avg, width, 
                 label=f'Global Average in {year}', color='#2ca02c')
    
    # Add labels, title, and custom x-axis tick labels
    ax.set_xlabel('COVID-19 Metrics', fontsize=10)
    ax.set_ylabel('Number of Cases', fontsize=10)
    ax.set_title(f'COVID-19 Statistics for {continent} in {year}', fontsize=14)
    ax.set_xticks(x)
    ax.set_xticklabels(categories, fontsize=10)
    ax.legend(fontsize=9)
    
    # Add counts above bars with NaN handling
    def add_labels(bars):
        for bar in bars:
            height = bar.get_height()
            label_text = f'{int(height):,}'
                
            ax.annotate(label_text,
                      xy=(bar.get_x() + bar.get_width() / 2, height if not np.isnan(height) else 0),
                      xytext=(0, 3),  # 3 points vertical offset
                      textcoords="offset points",
                      ha='center', va='bottom', fontsize=10)
    
    add_labels(bar1)
    add_labels(bar2)
    add_labels(bar3)
    
    # Add grid lines for better readability
    ax.grid(axis='y', linestyle='--', alpha=0.7)
    
    # Print the numerical results for reference
    print(f"{continent} values in {year}:")
    for category, value in zip(categories, continent_year_values):
        if np.isnan(value):
            print(f"  {category}: N/A")
        else:
            print(f"  {category}: {value:.0f}")
    
    print(f"\n{continent} yearly average across all time:")
    for category, value in zip(categories, continent_avg_values):
        if np.isnan(value):
            print(f"  {category}: N/A")
        else:
            print(f"  {category}: {value:.2f}")
    
    print(f"\nGlobal average in {year}:")
    for category, value in zip(categories, all_countries_year_avg):
        if np.isnan(value):
            print(f"  {category}: N/A")
        else:
            print(f"  {category}: {value:.2f}")
    
    plt.tight_layout()
    plt.show()

### Continent Total

In [15]:
def plot_continent_by_total(continent: str):
    df = pd.DataFrame(covid_19_dataset)

    # Get countries in the specified continent
    countries = regions_dataset[regions_dataset['Continent'] == continent]
    countries = countries['Country']
    
    # Convert Date column to datetime if needed
    if not pd.api.types.is_datetime64_any_dtype(df['Date']):
        df['Date'] = pd.to_datetime(df['Date'])
    
    # 1. Get all data for the specified continent
    continent_data = df[df['Country'].isin(countries)]
    
    # Calculate the total for the continent across all time
    continent_total_values = continent_data[['Confirmed', 'Recovered', 'Deaths']].sum().values
    
    # 2. Calculate global total for comparison
    global_total_values = df[['Confirmed', 'Recovered', 'Deaths']].sum().values
    
    # Set up the plot
    categories = ['Confirmed', 'Recovered', 'Deaths']
    x = np.arange(len(categories))
    width = 0.35  # Wider bars since we only have two
    
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Create the two sets of bars
    bar1 = ax.bar(x - width/2, continent_total_values, width, 
                 label=f'{continent} Total', color='#1f77b4')
    bar2 = ax.bar(x + width/2, global_total_values, width, 
                 label=f'Global Total', color='#2ca02c')
    
    # Add labels, title, and custom x-axis tick labels
    ax.set_xlabel('COVID-19 Metrics', fontsize=10)
    ax.set_ylabel('Number of Cases', fontsize=10)
    ax.set_title(f'COVID-19 Statistics Comparison: {continent} vs Global Total', fontsize=14)
    ax.set_xticks(x)
    ax.set_xticklabels(categories, fontsize=10)
    ax.legend(fontsize=10)
    
    # Add counts above bars with NaN handling
    def add_labels(bars):
        for bar in bars:
            height = bar.get_height()
            label_text = f'{int(height):,}'
                
            ax.annotate(label_text,
                      xy=(bar.get_x() + bar.get_width() / 2, height if not np.isnan(height) else 0),
                      xytext=(0, 3),  # 3 points vertical offset
                      textcoords="offset points",
                      ha='center', va='bottom', fontsize=8)
    
    add_labels(bar1)
    add_labels(bar2)
    
    # Add grid lines for better readability
    ax.grid(axis='y', linestyle='--', alpha=0.7)
    
    
    # Print the numerical results for reference
    print(f"{continent} total COVID-19 statistics:")
    for category, value in zip(categories, continent_total_values):
        if np.isnan(value):
            print(f"  {category}: N/A")
        else:
            print(f"  {category}: {value:.0f}")
    
    print(f"\nGlobal total COVID-19 statistics:")
    for category, value in zip(categories, global_total_values):
        if np.isnan(value):
            print(f"  {category}: N/A")
        else:
            print(f"  {category}: {value:.0f}")
    
    # Add a second y-axis to show the percentage
    ax2 = ax.twinx()
    ax2.set_ylabel('Percentage of Global Cases', fontsize=10)
    ax2.set_ylim(0, 100)  # Percentage scale
    
    plt.tight_layout()
    plt.show()

### World By Day

In [16]:
def plot_world_by_day(day: str):
    df = pd.DataFrame(covid_19_dataset)
    
    # Convert Date column to datetime
    df['Date'] = pd.to_datetime(df['Date'])
    
    # Target date
    target_date = pd.to_datetime(day)
    
    # 1. Get global data for the target date
    date_filter = df['Date'] == target_date
    target_df = df[date_filter]
    global_date_values = target_df[['Confirmed', 'Recovered', 'Deaths']].sum().values
    
    # 2. Calculate average for world across all days
    global_avg_values = df.groupby('Date')[['Confirmed', 'Recovered', 'Deaths']].sum().mean().values
    
    # Set up the plot
    categories = ['Confirmed', 'Recovered', 'Deaths']
    x = np.arange(len(categories))
    width = 0.35  # Wider bars since we only have two
    
    fig, ax = plt.subplots(figsize=(8, 5))
    
    # Create the two sets of bars
    bar1 = ax.bar(x - width/2, global_date_values, width, label=f'Global Avg. on {target_date.date()}', color='#1f77b4')
    bar2 = ax.bar(x + width/2, global_avg_values, width, label=f'Global Avg. (All Days)', color='#ff7f0e')
    
    # Add labels, title, and custom x-axis tick labels
    ax.set_xlabel('COVID-19 Metrics', fontsize=9)
    ax.set_ylabel('Number of Cases', fontsize=9)
    ax.set_title(f'COVID-19 Global Statistics Comparison for {target_date.date()}', fontsize=12)
    ax.set_xticks(x)
    ax.set_xticklabels(categories, fontsize=9)
    ax.legend(fontsize=9)
    
    # Add counts above bars
    def add_labels(bars):
        for bar in bars:
            height = bar.get_height()
            ax.annotate(f'{int(height):,}',
                        xy=(bar.get_x() + bar.get_width() / 2, height),
                        xytext=(0, 3),  # 3 points vertical offset
                        textcoords="offset points",
                        ha='center', va='bottom', fontsize=10)
    
    add_labels(bar1)
    add_labels(bar2)
    
    # Add grid lines for better readability
    ax.grid(axis='y', linestyle='--', alpha=0.7)
    
    # Print the numerical results for reference
    print(f"Global values on {target_date.date()}:")
    for category, value in zip(categories, global_date_values):
        print(f"  {category}: {int(value):,}")
    
    print(f"\nGlobal average across all days:")
    for category, value in zip(categories, global_avg_values):
        print(f"  {category}: {value:,.2f}")
    
    plt.tight_layout()
    plt.show()

### World By Month

In [17]:
def plot_world_by_month(month: str, year: str):
    df = pd.DataFrame(covid_19_dataset)
    
    # Convert Date column to datetime
    df['Date'] = pd.to_datetime(df['Date'])
    
    # Extract month and year for filtering
    month_num = int(month)
    year_num = int(year)
    
    # Add month and year columns for easy filtering
    df['Month'] = df['Date'].dt.month
    df['Year'] = df['Date'].dt.year
    
    # 1. Get global data for the target month
    month_filter = (df['Month'] == month_num) & (df['Year'] == year_num)
    target_df = df[month_filter]
    
    # Calculate the total global values for the month
    global_month_values = target_df[['Confirmed', 'Recovered', 'Deaths']].sum().values
    
    # 2. Calculate global average across all months
    # Group by month and year to get monthly averages
    global_avg_values = (df[['Confirmed', 'Recovered', 'Deaths']]
                         .groupby([df['Year'], df['Month']])
                         .sum()
                         .mean()
                         .values)
    
    # Set up the plot
    categories = ['Confirmed', 'Recovered', 'Deaths']
    x = np.arange(len(categories))
    width = 0.35  # Wider bars since we only have two
    
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Month name for display
    month_name = pd.to_datetime(f"{year}-{month}-01").strftime('%B')
    
    # Create the two sets of bars
    bar1 = ax.bar(x - width/2, global_month_values, width, 
                 label=f'Global Avg. in {month_name} {year}', color='#1f77b4')
    bar2 = ax.bar(x + width/2, global_avg_values, width, 
                 label=f'Global Monthly Avg. (All Time)', color='#ff7f0e')
    
    # Add labels, title, and custom x-axis tick labels
    ax.set_xlabel('COVID-19 Metrics', fontsize=10)
    ax.set_ylabel('Number of Cases', fontsize=10)
    ax.set_title(f'COVID-19 Global Statistics for {month_name} {year}', fontsize=14)
    ax.set_xticks(x)
    ax.set_xticklabels(categories, fontsize=10)
    ax.legend(fontsize=9)
    
    # Add counts above bars
    def add_labels(bars):
        for bar in bars:
            height = bar.get_height()
            ax.annotate(f'{int(height):,}',
                        xy=(bar.get_x() + bar.get_width() / 2, height),
                        xytext=(0, 3),  # 3 points vertical offset
                        textcoords="offset points",
                        ha='center', va='bottom', fontsize=10)
    
    add_labels(bar1)
    add_labels(bar2)
    
    # Add grid lines for better readability
    ax.grid(axis='y', linestyle='--', alpha=0.7)
    
    # Print the numerical results for reference
    print(f"Global values in {month_name} {year}:")
    for category, value in zip(categories, global_month_values):
        print(f"  {category}: {int(value):,}")
    
    print(f"\nGlobal monthly average across all time:")
    for category, value in zip(categories, global_avg_values):
        print(f"  {category}: {value:,.2f}")
    
    plt.tight_layout()
    plt.show()

### World By Year

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def plot_world_by_year(year: str):
    df = pd.DataFrame(covid_19_dataset)
    
    # Convert Date column to datetime
    df['Date'] = pd.to_datetime(df['Date'])
    
    # Extract year for filtering
    year_num = int(year)
    
    # Add year column for easy filtering
    df['Year'] = df['Date'].dt.year
    
    # 1. Get global data for the target year
    year_filter = df['Year'] == year_num
    target_df = df[year_filter]
    
    # Calculate the total global values for the year
    global_year_values = target_df[['Confirmed', 'Recovered', 'Deaths']].sum().values
    
    # 2. Calculate global average across all years
    global_avg_values = (df[['Confirmed', 'Recovered', 'Deaths']]
                         .groupby(df['Year'])
                         .sum()
                         .mean()
                         .values)
    
    # Set up the plot
    categories = ['Confirmed', 'Recovered', 'Deaths']
    x = np.arange(len(categories))
    width = 0.35  # Wider bars since we only have two
    
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Create the two sets of bars
    bar1 = ax.bar(x - width/2, global_year_values, width, 
                 label=f'Global Avg. in {year}', color='#1f77b4')
    bar2 = ax.bar(x + width/2, global_avg_values, width, 
                 label=f'Global Yearly Avg. (All Time)', color='#ff7f0e')
    
    # Add labels, title, and custom x-axis tick labels
    ax.set_xlabel('COVID-19 Metrics', fontsize=10)
    ax.set_ylabel('Number of Cases', fontsize=10)
    ax.set_title(f'COVID-19 Global Statistics for {year}', fontsize=14)
    ax.set_xticks(x)
    ax.set_xticklabels(categories, fontsize=10)
    ax.legend(fontsize=9)
    
    # Add counts above bars
    def add_labels(bars):
        for bar in bars:
            height = bar.get_height()
            ax.annotate(f'{int(height):,}',
                        xy=(bar.get_x() + bar.get_width() / 2, height),
                        xytext=(0, 3),  # 3 points vertical offset
                        textcoords="offset points",
                        ha='center', va='bottom', fontsize=10)
    
    add_labels(bar1)
    add_labels(bar2)
    
    # Add grid lines for better readability
    ax.grid(axis='y', linestyle='--', alpha=0.7)
    
    # Print the numerical results for reference
    print(f"Global values in {year}:")
    for category, value in zip(categories, global_year_values):
        print(f"  {category}: {int(value):,}")
    
    print(f"\nGlobal yearly average across all time:")
    for category, value in zip(categories, global_avg_values):
        print(f"  {category}: {value:,.2f}")
    
    plt.tight_layout()
    plt.show()


### World Total

In [19]:
def plot_world_by_total():
    df = pd.DataFrame(covid_19_dataset)
    
    # Convert Date column to datetime
    df['Date'] = pd.to_datetime(df['Date'])
    
    # Calculate the total global values for all time
    global_total_values = df[['Confirmed', 'Recovered', 'Deaths']].sum().values
    
    # Set up the plot
    categories = ['Confirmed', 'Recovered', 'Deaths']
    x = np.arange(len(categories))
    
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Create the total bar
    bar = ax.bar(x, global_total_values, color='#1f77b4', label='Global Total (All Time)')
    
    # Add labels, title, and custom x-axis tick labels
    ax.set_xlabel('COVID-19 Metrics', fontsize=10)
    ax.set_ylabel('Number of Cases', fontsize=10)
    ax.set_title('COVID-19 Global Total Statistics (All Time)', fontsize=14)
    ax.set_xticks(x)
    ax.set_xticklabels(categories, fontsize=10)
    ax.legend(fontsize=9)
    
    # Add counts above bars
    def add_labels(bars):
        for bar in bars:
            height = bar.get_height()
            ax.annotate(f'{int(height):,}',
                        xy=(bar.get_x() + bar.get_width() / 2, height),
                        xytext=(0, 3),  # 3 points vertical offset
                        textcoords="offset points",
                        ha='center', va='bottom', fontsize=10)
    
    add_labels(bar)
    
    # Add grid lines for better readability
    ax.grid(axis='y', linestyle='--', alpha=0.7)
    
    # Print the numerical results for reference
    print("Global total values (All Time):")
    for category, value in zip(categories, global_total_values):
        print(f"  {category}: {int(value):,}")
    
    plt.tight_layout()
    plt.show()

## Widget Events

### Regional Category Radio Button Event

In [20]:
def on_regional_category_change(change):
    index = change['new']
    # enable country and continent dropdown menus
    if index == 0:
        continent_dropdown_menu.disabled = False
        countries_dropdown_menu.disabled = False

    # disable country but enable continent dropdown menus
    elif index == 1:
        continent_dropdown_menu.disabled = False
        countries_dropdown_menu.disabled = True

    # disable both country and continent dropdown menus
    elif index == 2:
        continent_dropdown_menu.disabled = True
        countries_dropdown_menu.disabled = True

regional_category_radio.observe(on_regional_category_change, 'index')

### Continent Dropdown Menu Event

In [21]:
def on_continent_menu_change(change):
    # load countries of newly selected continent
    continent = change['new']
    continent_filter = regions_dataset['Continent'] == continent
    countries = regions_dataset[continent_filter]
    countries = countries['Country']
    countries_dropdown_menu.options = countries

continent_dropdown_menu.observe(on_continent_menu_change, 'value')

### Time Category Radio Button Event

In [22]:
def on_time_category_change(change):
    index = change['new']
    # enable day datepicker and disable year and month dropdown menus
    if index == 0:
        day_datepicker.disabled = False
        year_dropdown_menu.disabled = True
        month_dropdown_menu.disabled = True

    # disable day datepicker and enable year and month dropdown menus
    elif index == 1:
        day_datepicker.disabled = True
        year_dropdown_menu.disabled = False
        month_dropdown_menu.disabled = False

    # enable year drop down menu and disable both day datepicker and month dropdown menu
    elif index == 2:
        day_datepicker.disabled = True
        year_dropdown_menu.disabled = False
        month_dropdown_menu.disabled = True

    # disable all of day datepicker, year and month dropdown menus
    elif index == 3:
        day_datepicker.disabled = True
        year_dropdown_menu.disabled = True
        month_dropdown_menu.disabled = True

time_category_radio.observe(on_time_category_change, 'index')

### Reload Button Event

In [23]:
output = wgts.Output()
def on_reload_click(change):
    output.clear_output()
    time_category = time_category_radio.index
    regional_category = regional_category_radio.index
    
    # time_category = 0 --> daily
    if time_category == 0:
        # regional_category 0 --> country
        if regional_category == 0:
            with output:
                plot_country_by_day(countries_dropdown_menu.value, str(day_datepicker.value))
        # regional_category 1 --> continent
        elif regional_category == 1:
            with output:
                plot_continent_by_day(continent_dropdown_menu.value, str(day_datepicker.value))
        # regional_category 2 --> world
        elif regional_category == 2:
            with output:
                plot_world_by_day(str(day_datepicker.value))
    # time_category = 1 --> monthly
    if time_category == 1:
        month_num = datetime.datetime.strptime(month_dropdown_menu.value, "%B").month
        year_num = year_dropdown_menu.value
        # regional_category 0 --> country
        if regional_category == 0:
            with output:
                    plot_country_by_month(countries_dropdown_menu.value, month_num, year_num)
        # regional_category 1 --> continent
        elif regional_category == 1:
            with output:
                    plot_continent_by_month(continent_dropdown_menu.value, month_num, year_num)
        # regional_category 2 --> world
        elif regional_category == 2:
            with output:
                    plot_world_by_month(month_num, year_num)
    # time_category = 2 --> yearly
    if time_category == 2:
        # regional_category 0 --> country
        if regional_category == 0:
            with output:
                    plot_country_by_year(countries_dropdown_menu.value, year_dropdown_menu.value)
        # regional_category 1 --> continent
        elif regional_category == 1:
            with output:
                    plot_continent_by_year(continent_dropdown_menu.value, year_dropdown_menu.value)
        # regional_category 2 --> world
        elif regional_category == 2:
            with output:
                    plot_world_by_year(year_dropdown_menu.value)
    # time_category = 3 --> complete
    if time_category == 3:
        # regional_category 0 --> country
        if regional_category == 0:
            with output:
                    plot_country_by_total(countries_dropdown_menu.value)
        # regional_category 1 --> continent
        elif regional_category == 1:
            with output:
                    plot_continent_by_total(continent_dropdown_menu.value)
        # regional_category 2 --> world
        elif regional_category == 2:
            with output:
                    plot_world_by_total()

    
reload_button.on_click(on_reload_click)
wgts.VBox([reload_button, output, time_category_radio, day_datepicker, month_dropdown_menu, year_dropdown_menu, regional_category_radio, countries_dropdown_menu, continent_dropdown_menu])

## Layout

In [24]:
time_layout = wgts.HBox([time_category_radio, wgts.VBox([day_datepicker, month_dropdown_menu, year_dropdown_menu])])
region_layout = wgts.HBox([regional_category_radio, wgts.VBox([continent_dropdown_menu, countries_dropdown_menu])])

demo = wgts.VBox([time_layout, regional_category_radio, reload_button, output])

# Final Demo

In [25]:
demo